In [ ]:
import pandas as pd
from model_utils.utils_model import Predictor
from model_utils.util_global_struct import process_bb_old_to_new

In [ ]:
import numpy as np
import torch
from model_utils.utils_model import DataEncoder

In [ ]:
from scipy.special import softmax

In [ ]:
import plotly.express as px

In [ ]:
df = pd.read_pickle('data/human_transcriptome_segment_high_mfe_freq_testing_len64_100.pkl.gz')

In [ ]:
model_path = 's1_training/result/run_6/model_ckpt_ep_25.pth'  # still running

predictor = Predictor(model_ckpt=model_path,
                     num_filters=[32, 32, 64, 64, 64, 128, 128],
                     filter_width=[9, 9, 9, 9, 9, 9, 9],
                     dropout=0.0)


In [ ]:
idx = 99
seq = df.iloc[idx]['seq']
one_idx = df.iloc[idx]['one_idx']
bounding_boxes = df.iloc[idx]['bounding_boxes']
df_target = process_bb_old_to_new(bounding_boxes)

In [ ]:
seq

In [ ]:
df_target

In [ ]:
de = DataEncoder(seq)
yp = predictor.model(torch.tensor(de.x_torch))
yp = {k: v.detach().cpu().numpy()[0, :, :, :] for k, v in yp.items()}

In [ ]:
yp.keys()

In [ ]:
stem_on = yp['stem_on'][0, :, :]
# hard-mask lower triangular part
stem_on[np.tril_indices(len(seq))] = 0
fig = px.imshow(stem_on)
fig.update_layout(title='stem_on')

In [ ]:
yp['stem_on'][0, 29, 52]

In [ ]:
df_plot = pd.DataFrame()
for offset in range(-1, 4+1):  # bb size=4, we'll try offset -1, 0, 1, ...., 3, 4
    print(yp['stem_on'][0, 29, 55-offset])
    val = softmax(yp['stem_location_y'][:, 29, 55-offset])
    print(val)
    df_plot['location {}'.format(55-offset)] = val
    print(softmax(yp['stem_sm_size'][:, 29, 55-offset]))
    print(yp['stem_sl_size'][0, 29, 52-offset])
    print('')
fig = px.line(df_plot)
fig.update_layout(title='location_y', height=500, width=600)

In [ ]:
df_plot = pd.DataFrame()
for offset in range(-1, 4+1):  # bb size=4, we'll try offset -1, 0, 1, ...., 3, 4
    print(yp['stem_on'][0, 29+offset, 55])
    val = softmax(yp['stem_location_x'][:, 29+offset, 55])
    print(val)
    df_plot['location {}'.format(29+offset)] = val
#     print(softmax(yp['stem_sm_size'][:, 29, 55-offset]))
#     print(yp['stem_sl_size'][0, 29, 52-offset])
    print('')
fig = px.line(df_plot)
fig.update_layout(title='location_x', height=500, width=600)